In [1]:
"""


ABSOLUTELY NOTHING THAT WAS DONE WITH ADABOOST WAS VALUABLE BECAUSE OF A FUCK UP THAT CAUSED THE TRAINING TO BE GARBAGE

I'm leaving it all here just because I can

I will not be commenting or cleaning up any of it



"""

"\n\n\nABSOLUTELY NOTHING THAT WAS DONE WITH ADABOOST WAS VALUABLE BECAUSE OF A FUCK UP THAT CAUSED THE TRAINING TO BE GARBAGE\n\nI'm leaving it all here just because I can\n\nI will not be commenting or cleaning up any of it\n\n\n\n"

In [2]:
"""
Run once ever

Downloads and extracts the dataset
"""


# import urllib.request
# import tarfile
# import os

# mp3file = urllib.request.urlopen("https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz")
# with open('tmp','wb') as output:
#   output.write(mp3file.read())

# file = tarfile.open("tmp")
# file.extractall(".")
# file.close()
# os.remove("tmp")
# os.remove("cifar-100-python/file.txt~")

'\nRun once ever\n\nDownloads and extracts the dataset\n'

In [3]:

import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import cv2
from collections import Counter
from sklearn.metrics import confusion_matrix, zero_one_loss
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import threading
import time
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
#function to read files present in the Python version of the dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        myDict = pickle.load(fo, encoding='latin1')
    return myDict

In [5]:
trainData = unpickle('./cifar-100-python/train')
#type of items in each file
for item in trainData:
    print(item, type(trainData[item]))

filenames <class 'list'>
batch_label <class 'str'>
fine_labels <class 'list'>
coarse_labels <class 'list'>
data <class 'numpy.ndarray'>


In [6]:
testData = unpickle('./cifar-100-python//test')
metaData = unpickle('./cifar-100-python//meta')
#metaData
print("Fine labels:", metaData['fine_label_names'], "\n")
print("Coarse labels:", metaData['coarse_label_names'])

Fine labels: ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', '

In [7]:
#storing fine labels along with its number code in a dataframe
subCategory = pd.DataFrame(metaData['fine_label_names'], columns=['SubClass'])
print(subCategory)

         SubClass
0           apple
1   aquarium_fish
2            baby
3            bear
4          beaver
..            ...
95          whale
96    willow_tree
97           wolf
98          woman
99           worm

[100 rows x 1 columns]


In [8]:
# I'm pretty sure this is set up so that dimension 0 is the category
X_train = trainData['data']
X_train = X_train.reshape(len(X_train),3,32,32).transpose(0,2,3,1)

In [9]:
X_test = testData['data']
X_test = X_test.reshape(len(X_test),3,32,32).transpose(0,2,3,1)
#X_test

In [10]:
winSize = (32,32)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (2, 2)
nbins = 12


df = []

**Create Database of HOGs** 
(Pandas dataframe)

In [11]:
# this is essentially where we fit the model
DF = []
for i in range(len(X_train)):
# for i in range(5000):
    image = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    df.append(hog.compute(image))
DF = np.array(df)
df = []
for i in range(len(X_test)):
    image = cv2.cvtColor(X_test[i], cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    df.append(hog.compute(image))
DF_test = np.array(df)

In [12]:
DF[0]

array([0.        , 0.        , 0.00412205, ..., 0.00682359, 0.0028279 ,
       0.06439196], dtype=float32)

****continue work checkpoint: standardize test data and proceed****

In [13]:
def predict(img, k, winSize, blockSize, blockStride, cellSize, nbins):
    
    #Extract sample's features
    sample = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    _hog = hog.compute(sample)
    
    #Get Nearest Neighbors
    distances = []
    for i in range(len(DF)):
        distances.append(np.linalg.norm(_hog - DF[i]))         # 1- Calculate and store each datapoint's Euclidean distance
    # distances = np.linalg.norm((np.expand_dims(_hog, axis=0) - DF))
    
    kNNsID = np.argsort(distances)[:k]                              # 2- Sort distances' IDs in an ascending order
    
    kNNsCats = []
    for i in kNNsID:
        kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs
    
    CommonCat = Counter(kNNsCats).most_common(1)                    # 4- Count and get Most common category
    
    return CommonCat

In [14]:
n = 25
p = predict(X_test[n], 20, winSize, blockSize, blockStride, cellSize, nbins)

/tmp/ipykernel_24410/2374437631.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kNNsCats.append(subCategory.iloc[trainData['fine_labels'][i]][0].capitalize())      # 3- Get categories of each of the NNs


In [15]:
def getAccuracy(k, x_test, testdata):
    RightCount = 0
    preds = []
    for i in range(1000):
        pred = predict(x_test[i], k, winSize, blockSize, blockStride, cellSize, nbins)
        if pred[0][0] == subCategory.iloc[testdata['fine_labels'][i]][0].capitalize():
            RightCount += 1
        preds.append(pred)
        # print("Done: {}, Accuracy: {}, Time: {}".format(i,(RightCount/(i+1))*100, time.time() % 10000), flush=True)
    # it would be RightCount / 1000 * 100, so I just made it RightCount / 10
    return RightCount / 10

In [16]:
y_train = trainData["fine_labels"]
y_test = testData["fine_labels"]
X_train = trainData["data"]
X_test = testData["data"]

# X_train = np.array(pd.concat([pd.DataFrame(X_train), DF], axis=1))
# X_test = np.array(pd.concat([pd.DataFrame(X_test), DF_test], axis=1))
X_train = np.concat([X_train, DF], axis=1)
X_test = np.concat([X_test, DF_test], axis=1)

print(X_train.shape)
print(X_test.shape)

(50000, 12480)
(10000, 12480)


In [ ]:

# param_grid = {
#     # "nbins": [5, 7, 9, 11, 13],
#     "nbins": [13],
#     # "blockSize": [(2, 2), (4,4), (8,8)],
#     "blockSize": [(8,8)],
#     "blockStride": [(1,1), (2,2), (4,4), (8,8)],
#     "cellSize": [(1,1), (2,2), (4,4), (8,8)],
#     "k": [4, 7, 10, 14, 18, 25],
# }

# param_grid = {
#     "nbins": [10, 12],
#     # "blockSize": [(2, 2), (4,4), (8,8)],
#     "blockSize": [(8,8)],
#     "blockStride": [(1,1), (4,4)],
#     "cellSize": [(2,2)],
#     "k": [18, 20, 22, 25, 27, 30, 50],
# }

# list of configs that performed well with just 5k samples, but now testing with all of the samples
# param_grid = {
#     "nbins": [9, 11, 12],
#     "blockSize": [(8,8)],
#     "blockStride": [(4,4)],
#     "cellSize": [(2,2)],
#     "k": [10, 14, 18, 22, 25, 27],
#     # "k": [10],
# }
# adaBoost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), n_estimators=1000)
# adaBoost = AdaBoostClassifier()
# adaBoost.fit(X_train, y_train)

param_grid = {
    # "estimator": [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=4)],
    "depths": [1, 2, 3, 4],
    # "depths": [1, 2],
    # "n_estimators": [2500, 10000],
    # "n_estimators": [250, 500, 1000, 2500]
    "n_estimators": [10, 50, 100, 250, 500],
}

# print(X_train.shape)

# model = AdaBoostClassifier()

# 1103 minutes or 18 hours in, and only depth=2 was done, and depth=3 was ~half way through
# grid_search = GridSearchCV(model, param_grid, cv=10, scoring="accuracy", n_jobs=-1, verbose=2)
# grid_search.fit(DF, y_train)

# def train(d, n):
#     model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=d), n_estimators=n)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     print(f"Depth: {d}, n: {n} Accuracy: {1 - zero_one_loss(y_pred, y_test)}")


# threads = []
# for d in param_grid['depths']:
#     for n in param_grid['n_estimators']:
#         threads.append(threading.Thread(target=train, args=(d,n,)))
# for t in threads:
#     t.start()
# for t in threads:
#     t.join()

model = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=7), n_estimators=360)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Depth: {7}, n: {360} Accuracy: {1 - zero_one_loss(y_pred, y_test)}")




In [ ]:
# print(grid_search.cv_results_)
# print(grid_search.best_params_)

In [ ]:
TruePreds = []

for i in range(1000):
    TruePreds.append(subCategory.iloc[testData['fine_labels'][i]][0].capitalize())

/tmp/ipykernel_3124958/3796635586.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  TruePreds.append(subCategory.iloc[testData['fine_labels'][i]][0].capitalize())


In [ ]:
# Predicted = []
# for i in range(len(Preds14)):
#     Predicted.append(Preds14[i][0][0])

In [ ]:
# cm = confusion_matrix(TruePreds, Predicted)

In [ ]:
# x = np.array(TruePreds)
# len(np.unique(x))

In [ ]:
# cm_df = pd.DataFrame(cm,
#                      index = np.unique(x), 
#                      columns = np.unique(x))

In [ ]:
# plt.figure(figsize=(16,16))
# sns.heatmap(cm_df, annot=True)
# plt.title('Confusion Matrix')
# plt.ylabel('Actal Values')
# plt.xlabel('Predicted Values')
# plt.show()